In [28]:
import pandas as pd 
import json
import numpy as np 
import datetime
from random import randint

In [29]:
df=pd.read_excel("downloads/stochastic data2.xlsx")

In [30]:
df['Date']=pd.to_datetime(df['Date'])
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-12-15,3958.37,3958.37,3879.45,3895.75,3895.75,4493900000
1,2022-12-14,4015.54,4053.76,3965.65,3995.32,3995.32,4472340000
2,2022-12-13,4069.38,4100.96,3993.03,4019.65,4019.65,5079360000
3,2022-12-12,3939.29,3990.71,3935.30,3990.56,3990.56,3904130000
4,2022-12-09,3954.17,3977.02,3933.04,3934.38,3934.38,3888260000


In [31]:
new_set=[]
#we wil get 10000 sequences of data 
for row_set in range(100): 
    # we will generate two random numbers
    # first one which is the number of rows which will be taken 
    # the second one is the row from which we will start
    
    row_quant=randint(10,30)
    row_start=randint(0,len(df)-row_quant) 
# len(df)-row_quant means the maximum value we can get because
# if you get the maximum value so you have a 'quant_rows'to take into considration
# in another way if we said randint(0,len(df)) ,it maybe take take the value of len(df) 
# and then we won't have a plenty of rows to take into the data 

    market_subset=df.iloc[row_start:row_start+row_quant]
    Close_Date = max(market_subset['Date'])

    Close_Gap = market_subset['Close'].pct_change()
    High_Gap = market_subset['High'].pct_change()
    Low_Gap = market_subset['Low'].pct_change() 
    Volume_Gap = market_subset['Volume'].pct_change() 
    Daily_Change = (market_subset['Close'] - market_subset['Open']) / market_subset['Open']
    Outcome_Next_Day_Direction = (market_subset['Volume'].shift(-1) - market_subset['Volume'])
    
    new_set.append(pd.DataFrame({'Sequence_ID':[row_set+1]*len(market_subset),
                            'Close_Date':[Close_Date]*len(market_subset),
                           'Close_Gap':Close_Gap,
                           'High_Gap':High_Gap,
                           'Low_Gap':Low_Gap,
                           'Volume_Gap':Volume_Gap,
                           'Daily_Change':Daily_Change,
                           'Outcome_Next_Day_Direction':Outcome_Next_Day_Direction}))


In [32]:
new_set_df =pd.concat(new_set).dropna()
new_set_df 

,Sequence_ID,Close_Date,Close_Gap,High_Gap,Low_Gap,Volume_Gap,Daily_Change,Outcome_Next_Day_Direction
1051,1,2018-10-15,0.005940,-0.000079,-0.007126,0.209443,-0.001231,8.900800e+08
1052,1,2018-10-15,-0.014007,0.006978,-0.006935,0.221192,-0.017466,-3.598300e+08
1053,1,2018-10-15,0.021005,0.028220,0.027430,-0.073224,-0.030697,-1.022570e+09
1054,1,2018-10-15,0.033981,0.007241,0.032106,-0.224530,-0.000753,-1.498700e+08
1055,1,2018-10-15,0.001420,-0.001858,-0.004241,-0.042436,0.002398,-4.100000e+07
...,...,...,...,...,...,...,...,...
1068,100,2018-09-27,-0.007780,-0.007646,-0.005449,-0.030547,0.000464,-2.112700e+08
1069,100,2018-09-27,-0.001252,-0.000409,-0.004611,-0.063974,0.004694,-1.232100e+08
1070,100,2018-09-27,-0.005340,-0.002240,-0.001477,-0.039858,-0.005176,2.114300e+08
1071,100,2018-09-27,0.005601,0.001257,0.003330,0.071237,-0.000482,9.468000e+07


In [33]:
new_set_df['Close_Gap_LMH'] = pd.qcut(new_set_df['Close_Gap'], 3, labels=["L", "M", "H"])

# High_Gap - not used in this example
new_set_df['High_Gap_LMH'] = pd.qcut(new_set_df['High_Gap'], 3, labels=["L", "M", "H"])

# Low_Gap - not used in this example
new_set_df['Low_Gap_LMH'] = pd.qcut(new_set_df['Low_Gap'], 3, labels=["L", "M", "H"])

# Volume_Gap
new_set_df['Volume_Gap_LMH'] = pd.qcut(new_set_df['Volume_Gap'], 3, labels=["L", "M", "H"])
 
# Daily_Change
new_set_df['Daily_Change_LMH'] = pd.qcut(new_set_df['Daily_Change'], 3, labels=["L", "M", "H"])

# new set
new_set_df = new_set_df[["Sequence_ID", 
                         "Close_Date", 
                         "Close_Gap_LMH", 
                         "Volume_Gap_LMH", 
                         "Daily_Change_LMH", 
                         "Outcome_Next_Day_Direction"]]

new_set_df['Event_Pattern'] = new_set_df['Close_Gap_LMH'].astype(str) + new_set_df['Volume_Gap_LMH'].astype(str) + new_set_df['Daily_Change_LMH'].astype(str)

C:\Users\samer\AppData\Local\Temp/ipykernel_8688/3276375439.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_set_df['Event_Pattern'] = new_set_df['Close_Gap_LMH'].astype(str) + new_set_df['Volume_Gap_LMH'].astype(str) + new_set_df['Daily_Change_LMH'].astype(str)


In [34]:
# the data of one sequnece 
new_set_df.shape

(1750, 7)

In [35]:
compressed_set = new_set_df.groupby(['Sequence_ID', 
                                     'Close_Date'])['Event_Pattern'].apply(lambda x: "{%s}" % ', '.join(x)).reset_index()

print(compressed_set.shape)
compressed_set.head()
    

(100, 3)


,Sequence_ID,Close_Date,Event_Pattern
0,1,2018-10-15,"{HHM, LHL, HLL, HLM, MLM, MML, HHL, HMM, MLM, ..."
1,2,2022-10-28,"{LHL, HMM, HMH, LMH, LHH, LLL, HLM, HHL, LMH, ..."
2,3,2022-12-12,"{LML, HMH, LMM, MHL, HML, HLH, MHM, MHH, LLM, ..."
3,4,2020-05-20,"{LML, HHH, LLH, MMH, LHL, HLL, HLH, MMH, LHM, ..."
4,5,2022-02-07,"{HHH, LLL, HMH, LHH, LHH, LMH, LML, HHL, MLM, ..."


In [36]:
compressed_outcomes = new_set_df.groupby(['Sequence_ID', 'Close_Date'])['Outcome_Next_Day_Direction'].mean()
compressed_outcomes = compressed_outcomes.to_frame().reset_index()

In [37]:
compressed_outcomes

,Sequence_ID,Close_Date,Outcome_Next_Day_Direction
0,1,2018-10-15,-3.383731e+07
1,2,2022-10-28,6.631111e+06
2,3,2022-12-12,3.269909e+07
3,4,2020-05-20,2.525600e+07
4,5,2022-02-07,-2.627143e+07
...,...,...,...
95,96,2020-11-10,-4.288175e+08
96,97,2019-08-09,-4.574905e+07
97,98,2019-01-08,-2.257778e+07
98,99,2020-04-14,2.224964e+08


In [38]:
compressed_set = pd.merge(compressed_set, compressed_outcomes, on= ['Sequence_ID', 'Close_Date'], how='inner')
print(compressed_set.shape)
compressed_set.head()

(100, 4)


,Sequence_ID,Close_Date,Event_Pattern,Outcome_Next_Day_Direction
0,1,2018-10-15,"{HHM, LHL, HLL, HLM, MLM, MML, HHL, HMM, MLM, ...",-3.383731e+07
1,2,2022-10-28,"{LHL, HMM, HMH, LMH, LHH, LLL, HLM, HHL, LMH, ...",6.631111e+06
2,3,2022-12-12,"{LML, HMH, LMM, MHL, HML, HLH, MHM, MHH, LLM, ...",3.269909e+07
3,4,2020-05-20,"{LML, HHH, LLH, MMH, LHL, HLL, HLH, MMH, LHM, ...",2.525600e+07
4,5,2022-02-07,"{HHH, LLL, HMH, LHH, LHH, LMH, LML, HHL, MLM, ...",-2.627143e+07


In [39]:
compressed_set['Event_Pattern'] = [''.join(e.split()).replace('{','')
                                   .replace('}','') for e in compressed_set['Event_Pattern'].values]
compressed_set.head()

,Sequence_ID,Close_Date,Event_Pattern,Outcome_Next_Day_Direction
0,1,2018-10-15,"HHM,LHL,HLL,HLM,MLM,MML,HHL,HMM,MLM,MMM,MMM,ML...",-3.383731e+07
1,2,2022-10-28,"LHL,HMM,HMH,LMH,LHH,LLL,HLM,HHL,LMH,LML,HHH,LL...",6.631111e+06
2,3,2022-12-12,"LML,HMH,LMM,MHL,HML,HLH,MHM,MHH,LLM,MML,HLM,MH...",3.269909e+07
3,4,2020-05-20,"LML,HHH,LLH,MMH,LHL,HLL,HLH,MMH,LHM,LLL,HHM,LL...",2.525600e+07
4,5,2022-02-07,"HHH,LLL,HMH,LHH,LHH,LMH,LML,HHL,MLM,HHH,MLL,HL...",-2.627143e+07


In [40]:
compressed_set_validation = compressed_set[compressed_set['Close_Date'] >= datetime.datetime.now() 
                                           - datetime.timedelta(days=90)] # Sys.Date()-90 

compressed_set_validation.shape

(9, 4)

In [41]:
compressed_set = compressed_set[compressed_set['Close_Date'] < datetime.datetime.now() 
                                           - datetime.timedelta(days=90)]  
compressed_set.shape

(91, 4)

In [42]:
print(len(compressed_set['Outcome_Next_Day_Direction']))
len(compressed_set[abs(compressed_set['Outcome_Next_Day_Direction']) > 10000000])

91


73

In [43]:
# keep only keep big/interesting moves 
print('all moves:', len(compressed_set))
compressed_set = compressed_set[abs(compressed_set['Outcome_Next_Day_Direction']) > 10000000]
compressed_set
compressed_set['Outcome_Next_Day_Direction'] = np.where((compressed_set['Outcome_Next_Day_Direction'] > 0), 1, 0)
compressed_set_validation['Outcome_Next_Day_Direction'] = np.where((compressed_set_validation['Outcome_Next_Day_Direction'] > 0), 1, 0)
print('big moves only:', len(compressed_set))  

all moves: 91
big moves only: 73


In [44]:
compressed_set.head()

,Sequence_ID,Close_Date,Event_Pattern,Outcome_Next_Day_Direction
0,1,2018-10-15,"HHM,LHL,HLL,HLM,MLM,MML,HHL,HMM,MLM,MMM,MMM,ML...",0
3,4,2020-05-20,"LML,HHH,LLH,MMH,LHL,HLL,HLH,MMH,LHM,LLL,HHM,LL...",1
4,5,2022-02-07,"HHH,LLL,HMH,LHH,LHH,LMH,LML,HHL,MLM,HHH,MLL,HL...",0
6,7,2020-11-20,"HMH,MHL,HLM,HHH,LLH,LML,HLM,LHM,MHL,LLM,MMH,LM...",0
7,8,2021-08-26,"HHM,MHM,MLH,LMH,LHH,MLL,HML,HLH,MLM,MHM,MMM,MH...",1


In [45]:
compressed_set_pos = compressed_set[compressed_set['Outcome_Next_Day_Direction']==1][['Sequence_ID', 'Event_Pattern']]
print(compressed_set_pos.shape)
compressed_set_neg = compressed_set[compressed_set['Outcome_Next_Day_Direction']==0][['Sequence_ID', 'Event_Pattern']]
print(compressed_set_neg.shape)


(33, 2)
(40, 2)


In [46]:
compressed_set_pos.head()

,Sequence_ID,Event_Pattern
3,4,"LML,HHH,LLH,MMH,LHL,HLL,HLH,MMH,LHM,LLL,HHM,LL..."
7,8,"HHM,MHM,MLH,LMH,LHH,MLL,HML,HLH,MLM,MHM,MMM,MH..."
9,10,"MMM,MHM,MLM,MMM,MHH,LHL,HLH,LLL"
10,11,"LLH,MML,HLM,MLH,LLM,MHM,MHM,LLH,MHM,MLM,MHM,LL..."
11,12,"LHH,LLH,LHL,HLH,LHL,HLM,MMH,LHL,HLH,MML,HHM,LM..."


In [47]:
flat_list = [item.split(',') for item in compressed_set['Event_Pattern'].values ]
unique_patterns = ','.join(str(r) for v in flat_list for r in v)
unique_patterns = list(set(unique_patterns.split(',')))
len(unique_patterns)

27

In [48]:
unique_patterns

['HMH',
 'LHM',
 'LHL',
 'MHL',
 'LML',
 'MML',
 'LLM',
 'HLM',
 'MLL',
 'LLL',
 'HLL',
 'LLH',
 'LMM',
 'LMH',
 'HHM',
 'MHM',
 'MLH',
 'HML',
 'MHH',
 'HHH',
 'HMM',
 'MMM',
 'HLH',
 'MLM',
 'HHL',
 'LHH',
 'MMH']

Build the markov transition grid


In [26]:
def build_transition_grid(compressed_grid, unique_patterns):
    # build the markov transition grid

    patterns = []
    counts = []
    for from_event in unique_patterns:

        # how many times 
        for to_event in unique_patterns:
            pattern = from_event + ',' + to_event # MMM,MlM

            ids_matches = compressed_grid[compressed_grid['Event_Pattern'].str.contains(pattern)]
            found = 0
            if len(ids_matches) > 0:
                Event_Pattern = '---'.join(ids_matches['Event_Pattern'].values)
                found = Event_Pattern.count(pattern)
            patterns.append(pattern)
            counts.append(found)

    # create to/from grid
    grid_Df = pd.DataFrame({'pairs':patterns, 'counts': counts})

    grid_Df['x'], grid_Df['y'] = grid_Df['pairs'].str.split(',', 1).str
    grid_Df.head()

    grid_Df = grid_Df.pivot(index='x', columns='y', values='counts')

    grid_Df.columns= [col for col in grid_Df.columns]
    del grid_Df.index.name

    # replace all NaN with zeros
    grid_Df.fillna(0, inplace=True)
    grid_Df.head()

    #grid_Df.rowSums(transition_dataframe) 
    grid_Df = grid_Df / grid_Df.sum(1)
    return (grid_Df)

In [27]:
grid_pos = build_transition_grid(compressed_set_pos, unique_patterns) 
grid_neg = build_transition_grid(compressed_set_neg, unique_patterns) 

C:\Users\samer\AppData\Local\Temp/ipykernel_8688/2965077453.py:23: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  grid_Df['x'], grid_Df['y'] = grid_Df['pairs'].str.split(',', 1).str


AttributeError: can't delete attribute

In [1]:
grid_neg.head()


NameError: name 'grid_neg' is not defined

In [2]:
grid_pos.head()


NameError: name 'grid_pos' is not defined

In [3]:
# compressed_set_validation[compressed_set_validation['Sequence_ID' == seq_id]]


In [4]:
def safe_log(x,y):
   try:
      lg = np.log(x/y)
   except:
      lg = 0
   return lg

# predict on out of sample data
actual = []
predicted = []
for seq_id in compressed_set_validation['Sequence_ID'].values:
    patterns = compressed_set_validation[compressed_set_validation['Sequence_ID'] == seq_id]['Event_Pattern'].values[0].split(',')
    pos = []
    neg = []
    log_odds = []
    
    for id in range(0, len(patterns)-1):
        # get log odds
        # logOdds = log(tp(i,j) / tn(i,j)
        if (patterns[id] in list(grid_pos) and patterns[id+1] in list(grid_pos) and patterns[id] in list(grid_neg) and patterns[id+1] in list(grid_neg)):
                
            numerator = grid_pos[patterns[id]][patterns[id+1]]
            denominator = grid_neg[patterns[id]][patterns[id+1]]
            if (numerator == 0 and denominator == 0):
                log_value =0
            elif (denominator == 0):
                log_value = np.log(numerator / 0.00001)
            elif (numerator == 0):
                log_value = np.log(0.00001 / denominator)
            else:
                log_value = np.log(numerator/denominator)
        else:
            log_value = 0
        
        log_odds.append(log_value)
        
        pos.append(numerator)
        neg.append(denominator)
      
    print('outcome:', compressed_set_validation[compressed_set_validation['Sequence_ID']==seq_id]['Outcome_Next_Day_Direction'].values[0])
    print(sum(pos)/sum(neg))
    print(sum(log_odds))

    actual.append(compressed_set_validation[compressed_set_validation['Sequence_ID']==seq_id]['Outcome_Next_Day_Direction'].values[0])
    predicted.append(sum(log_odds))

from sklearn.metrics import confusion_matrix

confusion_matrix(actual, [1 if p > 0 else 0 for p in predicted])

NameError: name 'compressed_set_validation' is not defined